# **Crewai Chatbot**

In [ ]:
%pip install -Uq crewai crewai-tools crewai-tools langchain langchain_community google-generativeai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['GEMINI_API_KEY'] =  os.getenv('GEMINI_API_KEY')
os.environ['SERPER_API_KEY'] =  os.getenv('SERPAPI_API_KEY')

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

e:\Rida\PIAIC\Quarter4\Crewai Bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": os.getenv('GEMINI_API_KEY'),
         }
}

In [6]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource

# Create a PDF knowledge source
pdf_source = PDFKnowledgeSource(
    file_paths=["tax-card.pdf"]
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[pdf_source], # Enable knowledge by adding the sources here
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "Where the holding period exceeds 4 Years but does not exceed 5 Years"})

# Agent: About User
## Task: Answer the following questions about the user: Where the holding period exceeds 4 Years but does not exceed 5 Years


# Agent: About User
## Final Answer: 
Where the holding period exceeds 4 Years but does not exceed 5 Years:

*   **Capital Gains on Disposal of Immoveable Property (Acquired between 1 Jul 2022 to 30 Jun 2024):**
    *   Constructed Property: 5%
    *   Open Plots: 0%
    *   Flats: - (This likely indicates that the rate is not specified or doesn't exist for flats in this holding period)

*   **Capital Gains on Disposal of Securities (Acquired between 1 Jul 2022 to 30 Jun 2024):** 5% for Non-Filer with the cap rate of 15%




In [11]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'

In [44]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),

    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

Overriding of current TracerProvider is not allowed


In [7]:
embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": os.getenv('GEMINI_API_KEY'),
        }
    }

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

print("API Key:", os.getenv("SERPAPI_API_KEY"))

API Key: 0937136758d3619a8c8aee168aaa0642d0aca58f


In [13]:
from pydantic import Field
from crewai.tools import BaseTool
from crewai import Agent, Task, Crew
from langchain_community.tools import YouTubeSearchTool
from langchain_community.utilities import GoogleSerperAPIWrapper

from dotenv import load_dotenv
load_dotenv()

search = GoogleSerperAPIWrapper()

class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: GoogleSerperAPIWrapper = Field(default_factory=GoogleSerperAPIWrapper)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"


# Create agents
llm_agent = Agent(
    role='Student Helper',
    goal='Answer students’ questions or search the web/YouTube if unsure.',
    backstory='A knowledgeable AI assistant who helps students with their queries and provides relevant resources.',
    tools=[SearchTool()],
    verbose=True,
    llm=llm1
)

# Define tasks
def student_query_task(question):
    return Task(
        description=f"Answer the student query or search online if needed: {question}",
        expected_output='A helpful answer to the student’s question, with links to additional resources if available.',
        agent=llm_agent
    )

# Assemble the crew
def run_chatbot(question):
    task = student_query_task(question)
    crew = Crew(
        agents=[llm_agent],
        tasks=[task],
        verbose=True,
        planning=True,
        planning_llm=llm1,
        embedder=google_embedder
    )
    result = crew.kickoff(inputs={"question": question})
    return result

# Example query
question = "Explain boyle's laws of motion and share a YouTube video about it."
response = run_chatbot(question)
print("Bot's Response:", response)

Overriding of current TracerProvider is not allowed


 
[2025-03-05 15:15:17][INFO]: Planning the crew execution
# Agent: Student Helper
## Task: Answer the student query or search online if needed: Explain boyle's laws of motion and share a YouTube video about it.1. **Search for Boyle's Law Explanation:** The Student Helper will use the 'Search' tool with the query "Boyle's Law explained" to gather information about Boyle's Law from reputable sources.
2. **Synthesize Information:** The Student Helper will synthesize the information obtained from the search results to create a clear and concise explanation of Boyle's Law. This explanation should cover the relationship between pressure and volume of a gas at constant temperature.
3. **Search for Relevant YouTube Video:** The Student Helper will use the 'Search' tool again with the query "Boyle's Law YouTube video" to find a suitable video that explains Boyle's Law in an understandable way.
4. **Select and Review Video:** The Student Helper will review the top search results for YouTube vid